In [1]:
using Revise

In [330]:
using LearningMPC
using LearningMPC.Models
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using RigidBodyDynamics.PDControl
using MomentumBasedControl
using MeshCatMechanisms
using RigidBodySim
using RigidBodySim.Visualization.MeshCatInterface
import LCPSim
using JLD2

In [333]:
robot = BoxAtlas();

In [1]:
using JuMP

In [338]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [339]:
rootframe = root_frame(mechanism(robot))
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism(robot), floor)
wall = HalfSpace3D(Point3D(rootframe, 0., 1.0, 0.), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism(robot), wall)

2-element Array{RigidBodyDynamics.Contact.HalfSpace3D{Float64},1}:
 RigidBodyDynamics.Contact.HalfSpace3D{Float64}(Point3D in "world": [0.0, 0.0, 0.0], FreeVector3D in "world": [0.0, 0.0, 1.0]) 
 RigidBodyDynamics.Contact.HalfSpace3D{Float64}(Point3D in "world": [0.0, 1.0, 0.0], FreeVector3D in "world": [0.0, -1.0, 0.0])

In [340]:
feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(robot), "l_hand_mount"), :right => findbody(mechanism(robot), "r_hand_mount"));

contact_model = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))

for (side, body) in feet
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end

for (side, body) in hands
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end


In [342]:
momentum_based_controller = MomentumBasedController(mechanism(robot))
contacts = add_mechanism_contacts!(momentum_based_controller)

Dict{RigidBodyDynamics.RigidBody{Float64},Array{MomentumBasedControl.ContactSettings,1}} with 14 entries:
  l_hand_mount_dummy  => MomentumBasedControl.ContactSettings[]
  l_hand_mount        => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  r_foot_sole_dummy   => MomentumBasedControl.ContactSettings[]
  l_foot_sole_dummy   => MomentumBasedControl.ContactSettings[]
  pelvis              => MomentumBasedControl.ContactSettings[]
  root                => MomentumBasedControl.ContactSettings[]
  floating_base_dummy => MomentumBasedControl.ContactSettings[]
  wall                => MomentumBasedControl.ContactSettings[]
  r_hand_mount_dummy  => MomentumBasedControl.ContactSettings[]
  floor               => MomentumBasedControl.ContactSettings[]
  world               => MomentumBasedControl.ContactSettings[]
  r_foot_sole         => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  r_hand_mount        => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  l_foot_so

In [343]:
μ = 0.8
floor_normal = floor.outward_normal
wall_normal = wall.outward_normal

FreeVector3D in "world": [0.0, -1.0, 0.0]

In [345]:
# FIXME: referencing global variables
function lqr_high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(momentum_based_controller)
    MomentumBasedControl.reset!(momentum_based_controller)
    regularize_joint_accels!(momentum_based_controller, 0.05)
    
    for (side, body) in feet
        for contactsettings in contacts[body]
            set!(contactsettings, 1e-4, μ, transform(state, floor_normal, default_frame(body)))
        end
    end
    
    momentum_based_controller.terminalstatetask.x .= lqrsol.x0
    momentum_based_controller.terminalstatetask.Q .= lqrsol.S
    momentum_based_controller.terminalstatetask.q .= 0
    
    # Low level control
    momentum_based_controller(τ, t, state)
    τ
end

lqr_high_level_control (generic function with 1 method)

In [412]:
# FIXME: referencing global variables
function net_high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(momentum_based_controller)
    MomentumBasedControl.reset!(momentum_based_controller)
    regularize_joint_accels!(momentum_based_controller, 0.05)
    
    for (side, body) in feet
        for contactsettings in contacts[body]
            set!(contactsettings, 1e-4, μ, transform(state, floor_normal, default_frame(body)))
        end
    end
    
    body = hands[:left]
    for contactsettings in contacts[body]
        set!(contactsettings, 1e-1, μ, transform(state, wall_normal, default_frame(body)))
    end
    
    momentum_based_controller.terminalstatetask.x .= lqrsol.x0
    momentum_based_controller.terminalstatetask.Q .= lqrsol.Q
    q0, q = net_cost.tangent_net(vcat(configuration(state), velocity(state)))
    momentum_based_controller.terminalstatetask.q .= vec(Flux.Tracker.data(q))
#     momentum_based_controller.terminalstatetask.Q .= 0
#     momentum_based_controller.terminalstatetask.q .= 0
    
    # Low level control
    momentum_based_controller(τ, t, state)
    τ
end

net_high_level_control (generic function with 1 method)

In [409]:
effort_bounds = LCPSim.all_effort_bounds(mechanism(robot))

function effort_limiter(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= clamp.(τ, effort_bounds)
    τ
end

position_bounds = LCPSim.all_configuration_bounds(mechanism(robot))

function bounds_enforcer(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: handle q̇ vs v correctly
    for i in 1:num_positions(state)
        kp = 1000
        kd = 0.1 * kp
        if configuration(state)[i] > position_bounds[i].upper
            violation = configuration(state)[i] - position_bounds[i].upper
            τ[i] -= kp * violation
            τ[i] -= kd * velocity(state)[i]
        elseif configuration(state)[i] < position_bounds[i].lower
            violation = position_bounds[i].lower - configuration(state)[i]
            τ[i] += kp * violation 
            τ[i] -= kd * velocity(state)[i]
        end
    end
    τ
end

function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end

    

compose (generic function with 1 method)

In [399]:
mvis = MechanismVisualizer(robot)
MeshCat.IJuliaCell(mvis)

WebIO.Scope("scope-6c550f6c-125f-4e17-9097-2d8a3362a622", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("height", "100%"),Pair{String,Any}("scrolling", "no"),Pair{String,Any}("src", "javascript:void(0)"),Pair{String,Any}("frameborder", 0))),Pair{Symbol,Any}(:id, "ifr"),Pair{Symbol,Any}(:style, Dict("width"=>"100%","flexGrow"=>"1"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:style, Dict("flexDirection"=>"column","height"=>"100%","minHeight"=>"400px","display"=>"flex","width"=>"100%","overflow"=>"hidden"))), 1), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(), Set{String}(), nothing, Any[], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function () {\n    var frame = this.dom.querySelector(\"#ifr\");\n    var doc = frame.contentDocument\n    var win = frame.contentWindow\n\n    // Determine if we're running on a Jupyter hosting service\n    // that requires a base URL when retrieving assets\n    var curMatch = \n        window.location.href\n        .match(/(.*)\\/notebooks\\/.*\\.ipynb/);\n    curMatch = curMatch ||\n        window.location.href\n        .match(/(.*)\\/apps\\/.*\\.ipynb/);\n    if (curMatch) {\n        var base = doc.createElement(\"base\");\n        base.setAttribute(\"href\", curMatch[1] + '/');\n        doc.head.appendChild(base);\n    }\n\n    var webio = doc.createElement(\"script\")\n    webio.src = \"pkg/WebIO/webio/dist/bundle.js\"\n    var parent = window\n\n    function resizeIframe() {\n        doc.body.style.padding = '0'\n        doc.body.style.margin = '0'\n        doc.documentElement.height = '100%'\n        doc.body.height = '100%'\n    }\n\n    webio.onload = function () {\n        win.WebIO.sendCallback = parent.WebIO.sendCallback; // Share stuff\n        win.WebIO.scopes = parent.WebIO.scopes\n        win.WebIO.obsscopes = parent.WebIO.obsscopes\n        win.WebIO._connected = true\n        doc.body.innerHTML = \"<html><body>\" + \"<div class='display:none'></div><unsafe-script style='display:none'>\\nWebIO.mount(this.previousSibling,{&quot;props&quot;:{},&quot;nodeType&quot;:&quot;Scope&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;imports&quot;:{&quot;data&quot;:[{&quot;name&quot;:null,&quot;type&quot;:&quot;js&quot;,&quot;url&quot;:&quot;pkg/MeshCat/meshcat/dist/main.min.js&quot;}],&quot;type&quot;:&quot;async_block&quot;},&quot;id&quot;:&quot;scope-8617b1f1-3872-44df-980d-ce6866de15b2&quot;,&quot;handlers&quot;:{&quot;meshcat-command&quot;:[(function (val){return this.viewer.handle_command_message({&quot;data&quot;:val})})],&quot;_promises&quot;:{&quot;importsLoaded&quot;:[(function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({&quot;name&quot;:&quot;meshcat-request&quot;,&quot;scope&quot;:&quot;scope-8617b1f1-3872-44df-980d-ce6866de15b2&quot;,&quot;id&quot;:&quot;ob_29&quot;,&quot;type&quot;:&quot;observable&quot;},String(Date.now())); return window.document.body.style.margin=&quot;0&quot;})]}},&quot;systemjs_options&quot;:null,&quot;observables&quot;:{&quot;meshcat-command&quot;:{&quot;sync&quot;:false,&quot;id&quot;:&quot;ob_28&quot;,&quot;value&quot;:[131,166,109,97,116,114,105,120,199,64,23,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,164,116,121,112,101,173,115,101,116,95,116,114,97,110,115,102,111,114,109,164,112,97,116,104,218,0,76,47,109,101,115,104,99,97,116,47,119,111,114,108,100,47,114,111,111,116,47,102,108,111,97,116,105,110,103,95,98,97,115,101,95,100,117,109,109,121,47,112,101,108,118,105,115,47,108,95,102,111,111,116,95,115,111,108,101,95,100,117,109,109,121,47,108,95,102,111,111,116,95,115,111,108,101]},&quot;meshcat-controls&quot;:{&quot;sync&quot;:true,&quot;id&quot;:&quot;ob_30&quot;,&quot;value&q

In [413]:
state = nominal_state(robot)
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [-0.5, 0, 0.])
final_time = 5.
Δt = 0.01

composed = PeriodicController(similar(velocity(state)), Δt, compose(net_high_level_control, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer)

dynamics = Dynamics(mechanism(robot), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback, CallbackSet(mvis, state, max_fps = 30.)))

@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = Δt);

 11.642648 seconds (16.25 M allocations: 1.103 GiB, 8.86% gc time)


In [414]:
RigidBodySim.animate(mvis, state, sol)